In [ ]:
!pip install numpy

In [ ]:
import tensorflow as tf
import os

In [ ]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(origin=_URL, fname='flower_photos.tgz', extract=True, cache_subdir='/content')

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228827136/228813984 [==============================] - 2s 0us/step


In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training'
  )

val_generator = datagen.flow_from_directory(
     base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10

history = model.fit(train_generator,
                    epochs = epochs,
                    validation_data= val_generator)

Epoch 1/10
46/46 [==============================] - 111s 2s/step - loss: 0.8492 - accuracy: 0.7186 - val_loss: 0.4740 - val_accuracy: 0.8440
Epoch 2/10
46/46 [==============================] - 106s 2s/step - loss: 0.3148 - accuracy: 0.8925 - val_loss: 0.4189 - val_accuracy: 0.8482
Epoch 3/10
46/46 [==============================] - 108s 2s/step - loss: 0.2023 - accuracy: 0.9319 - val_loss: 0.4337 - val_accuracy: 0.8495
Epoch 4/10
46/46 [==============================] - 129s 3s/step - loss: 0.1388 - accuracy: 0.9534 - val_loss: 0.4512 - val_accuracy: 0.8618
Epoch 5/10
46/46 [==============================] - 108s 2s/step - loss: 0.0909 - accuracy: 0.9724 - val_loss: 0.4564 - val_accuracy: 0.8564
Epoch 6/10
46/46 [==============================] - 107s 2s/step - loss: 0.0551 - accuracy: 0.9898 - val_loss: 0.4811 - val_accuracy: 0.8618
Epoch 7/10
46/46 [==============================] - 108s 2s/step - loss: 0.0314 - accuracy: 0.9956 - val_loss: 0.4858 - val_accuracy: 0.8632
Epoch 8/10
46

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>